In [1]:
import numpy as np
import pandas as pd
import pickle
import geopandas as gpd
from shapely.geometry import Point

import json

import netCDF4
 
import datetime

import psycopg2

import os


# AGGREGATE

In [2]:
# load level 2 dict
with open('input/dicts/adm_2_to_grid.pkl', 'rb') as handle:
    adm_2_to_grid = pickle.load(handle)

In [3]:
print(len(adm_2_to_grid.keys()))
print(len(adm_2_to_grid["ITA"].keys()))
adm_2_to_grid["ITA"].keys()

234
20


dict_keys(['ITA.15_1', 'ITA.4_1', 'ITA.14_1', 'ITA.2_1', 'ITA.3_1', 'ITA.5_1', 'ITA.8_1', 'ITA.12_1', 'ITA.1_1', 'ITA.16_1', 'ITA.18_1', 'ITA.11_1', 'ITA.9_1', 'ITA.6_1', 'ITA.13_1', 'ITA.10_1', 'ITA.20_1', 'ITA.19_1', 'ITA.7_1', 'ITA.17_1'])

In [4]:
#Define date range
start = datetime.date(2020, 1, 1)
stop = datetime.date(2020, 4, 30)
step = datetime.timedelta(days=1)
DATERANGE = pd.date_range(start, stop, freq=step)
DATERANGE


DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10',
               ...
               '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24',
               '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28',
               '2020-04-29', '2020-04-30'],
              dtype='datetime64[ns]', length=121, freq='D')

## Create a DF for each variable and then merge

In [2]:
variables = {'precip_max': {'folder': 'precip_max', 
                            'file': 'global_daily_precip_max_',
                            'variable': 'precipitation_flux'},
            'precip_mean': {'folder': 'precip_mean', 
                            'file': 'global_daily_precip_mean_',
                            'variable': 'precipitation_flux'},
            'specific_humidity_max': {'folder': 'sh_max', 
                                      'file': 'global_daily_sh_max_',
                                      'variable': 'specific_humidity'},
            'specific_humidity_mean': {'folder': 'sh_mean', 
                                       'file': 'global_daily_sh_mean_',
                                       'variable': 'specific_humidity'},
            'specific_humidity_min': {'folder': 'sh_min', 
                                      'file': 'global_daily_sh_min_',
                                      'variable': 'specific_humidity'},
            'short_wave_radiation_max': {'folder': 'sw_max', 
                                         'file': 'global_daily_sw_max_',
                                         'variable': 'm01s01i202'},
            'short_wave_radiation_mean': {'folder': 'sw_mean', 
                                          'file': 'global_daily_sw_mean_',
                                          'variable': 'm01s01i202'},
            'air_temperature_max': {'folder': 't1o5m_max', 
                                     'file': 'global_daily_t1o5m_max_',
                                     'variable': 'air_temperature'},
            'air_temperature_mean': {'folder': 't1o5m_mean', 
                                     'file': 'global_daily_t1o5m_mean_',
                                     'variable': 'air_temperature'},
            'air_temperature_min': {'folder': 't1o5m_min', 
                                     'file': 'global_daily_t1o5m_min_',
                                     'variable': 'air_temperature'},
            'windgust_max': {'folder': 'windgust_max', 
                                     'file': 'global_daily_windgust_max_',
                                     'variable': 'wind_speed_of_gust'},
            'windgust_mean': {'folder': 'windgust_mean', 
                                     'file': 'global_daily_windgust_mean_',
                                     'variable': 'wind_speed_of_gust'},
            'windgust_min': {'folder': 'windgust_min', 
                                     'file': 'global_daily_windgust_min_',
                                     'variable': 'wind_speed_of_gust'},
             
            'windspeed_max': {'folder': 'windspeed_max', 
                                     'file': 'global_daily_windspeed_max_',
                                     'variable': 'wind_speed'},
            'windspeed_mean': {'folder': 'windspeed_mean', 
                                     'file': 'global_daily_windspeed_mean_',
                                     'variable': 'wind_speed'},
            'windspeed_min': {'folder': 'windspeed_min', 
                                     'file': 'global_daily_windspeed_min_',
                                     'variable': 'wind_speed'}
             
            }

In [6]:
def create_aggr_df(indicator, daterange):
    days = []
    country = []
    avg = []
    std = []
    region = []
    city = []

    print("loading data for {} from {} to {}".format(indicator, daterange[0], daterange[-1]))
    
    for day in daterange:
        nc = netCDF4.Dataset("data/metoffice_global_daily/{}/{}{}.nc".format(variables[indicator]['folder'],
                                                variables[indicator]['file'],
                                                day.strftime('%Y%m%d')))

        data = nc.variables[variables[indicator]['variable']][:].data.reshape(-1)

        for area_0 in adm_2_to_grid:
            for area_1 in adm_2_to_grid[area_0]:
                for area_2 in adm_2_to_grid[area_0][area_1]:
                    idx_list = [point[0] for point in adm_2_to_grid[area_0][area_1][area_2]]

                    to_avg = [data[idx] for idx in idx_list]

                    days.append(day.strftime('%Y-%m-%d'))
                    country.append(area_0)
                    region.append(area_1)
                    city.append(area_2)
                    avg.append(np.mean(to_avg))
                    std.append(np.std(to_avg))
        
    d = {'day': days, 'country': country, 'region': region, 'city': city,
         indicator+'_avg': avg, 
         indicator+'_std': std }
    return pd.DataFrame(data=d)
    

In [7]:
%%time
dfs = [create_aggr_df(indicator, DATERANGE) for indicator in variables]

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31

2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19

2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08

In [8]:
dfs[-1]

,day,country,region,city,windspeed_min_avg,windspeed_min_std
0,2020-01-01,LVA,LVA.2_1,LVA.2.2_1,3.808333,0.334996
1,2020-01-01,LVA,LVA.2_1,LVA.2.3_1,3.814815,0.275064
2,2020-01-01,LVA,LVA.2_1,LVA.2.5_1,3.567708,0.210342
3,2020-01-01,LVA,LVA.2_1,LVA.2.6_1,3.837963,0.309597
4,2020-01-01,LVA,LVA.2_1,LVA.2.4_1,3.836207,0.222891
...,...,...,...,...,...,...
4610700,2020-04-30,GHA,GHA.7_1,GHA.7.3_1,0.900000,0.357071
4610701,2020-04-30,GHA,GHA.7_1,GHA.7.2_1,1.812500,0.139754
4610702,2020-04-30,GHA,GHA.7_1,GHA.7.6_1,1.229167,0.384712
4610703,2020-04-30,GHA,GHA.7_1,GHA.7.1_1,1.604167,0.522497


In [9]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on=['day', 'country', 'region', 'city']), dfs)

In [10]:
df_final

,day,country,region,city,precip_max_avg,precip_max_std,precip_mean_avg,precip_mean_std,specific_humidity_max_avg,specific_humidity_max_std,...,windgust_mean_avg,windgust_mean_std,windgust_min_avg,windgust_min_std,windspeed_max_avg,windspeed_max_std,windspeed_mean_avg,windspeed_mean_std,windspeed_min_avg,windspeed_min_std
0,2020-01-01,LVA,LVA.2_1,LVA.2.2_1,0.000011,0.000003,9.536743e-07,3.433620e-07,0.004004,0.000120,...,9.530557,0.520522,7.308333,0.499722,6.337500,0.329378,4.898785,0.352641,3.808333,0.334996
1,2020-01-01,LVA,LVA.2_1,LVA.2.3_1,0.000016,0.000004,1.783724e-06,6.993254e-07,0.003906,0.000000,...,10.395448,0.458968,7.291667,0.472631,7.023148,0.252389,5.353395,0.268541,3.814815,0.275064
2,2020-01-01,LVA,LVA.2_1,LVA.2.5_1,0.000015,0.000004,1.801385e-06,5.537813e-07,0.004110,0.000091,...,9.373916,0.297958,7.041667,0.324091,6.093750,0.405448,4.665581,0.207931,3.567708,0.210342
3,2020-01-01,LVA,LVA.2_1,LVA.2.6_1,0.000019,0.000006,2.166371e-06,8.423381e-07,0.004042,0.000121,...,10.145833,0.371016,7.250000,0.540062,6.425926,0.481437,5.242283,0.224034,3.837963,0.309597
4,2020-01-01,LVA,LVA.2_1,LVA.2.4_1,0.000013,0.000004,1.293489e-06,4.332328e-07,0.003982,0.000113,...,10.665770,0.413598,7.215517,0.375569,6.836207,0.389937,5.593211,0.271090,3.836207,0.222891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610700,2020-04-30,GHA,GHA.7_1,GHA.7.3_1,0.000858,0.000332,6.837845e-05,3.005889e-05,0.023877,0.002108,...,6.887500,0.178505,2.000000,0.500000,5.075000,0.127476,3.056250,0.099172,0.900000,0.357071
4610701,2020-04-30,GHA,GHA.7_1,GHA.7.2_1,0.002724,0.001966,1.564821e-04,1.043309e-04,0.021362,0.000273,...,7.388021,0.147705,1.812500,0.840851,5.343750,0.103645,3.382812,0.056876,1.812500,0.139754
4610702,2020-04-30,GHA,GHA.7_1,GHA.7.6_1,0.002940,0.001452,1.405080e-04,5.828643e-05,0.023153,0.002792,...,7.279514,0.199479,2.645833,1.019131,4.812500,0.200909,3.338542,0.062717,1.229167,0.384712
4610703,2020-04-30,GHA,GHA.7_1,GHA.7.1_1,0.004011,0.001130,2.075831e-04,4.245686e-05,0.022054,0.000336,...,7.259549,0.147847,2.895833,0.785337,5.187500,0.119678,3.340278,0.108531,1.604167,0.522497


In [11]:
!mkdir to_upload
df_final.to_pickle("to_upload/WORLD_20200101_20200430.pkl")

mkdir: to_upload: File exists


# Upload to DB

In [12]:
import pandas as pd
import json
import psycopg2

In [2]:
def create_weather_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS weather (
            day text NOT NULL,
            countrycode text NOT NULL,
            gid text NOT NULL,
            precip_max_avg float,
            precip_max_std float,
            precip_mean_avg float,
            precip_mean_std float,
            specific_humidity_max_avg float,
            specific_humidity_max_std float,
            specific_humidity_mean_avg float,
            specific_humidity_mean_std float,
            specific_humidity_min_avg float,
            specific_humidity_min_std float,
            short_wave_radiation_max_avg float,
            short_wave_radiation_max_std float,
            short_wave_radiation_mean_avg float,
            short_wave_radiation_mean_std float,
            air_temperature_max_avg float,
            air_temperature_max_std float,
            air_temperature_mean_avg float,
            air_temperature_mean_std float,
            air_temperature_min_avg float,
            air_temperature_min_std float,
            windgust_max_avg float,
            windgust_max_std float,
            windgust_mean_avg float,
            windgust_mean_std float,
            windgust_min_avg float,
            windgust_min_std float,
            windspeed_max_avg float,
            windspeed_max_std float,
            windspeed_mean_avg float,
            windspeed_mean_std float,
            windspeed_min_avg float,
            windspeed_min_std float,

            UNIQUE (gid, day),
            PRIMARY KEY (gid, day)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("weather table created")

In [3]:
def drop_weather_table(cur):
    sql_drop_table = """DROP TABLE weather"""
    
    try:
        cur.execute(sql_drop_table)
        print("weather table deleted")
    except:
        print("weather not present")

In [8]:
def import_data(cur,data):
    
    INSERT_STATEMENT = 'INSERT INTO weather \
                        (day, \
                        countrycode, \
                        gid, \
                        precip_max_avg, \
                        precip_max_std, \
                        precip_mean_avg, \
                        precip_mean_std, \
                        specific_humidity_max_avg, \
                        specific_humidity_max_std, \
                        specific_humidity_mean_avg, \
                        specific_humidity_mean_std, \
                        specific_humidity_min_avg, \
                        specific_humidity_min_std, \
                        short_wave_radiation_max_avg, \
                        short_wave_radiation_max_std, \
                        short_wave_radiation_mean_avg, \
                        short_wave_radiation_mean_std, \
                        air_temperature_max_avg, \
                        air_temperature_max_std, \
                        air_temperature_mean_avg, \
                        air_temperature_mean_std, \
                        air_temperature_min_avg, \
                        air_temperature_min_std, \
                        windgust_max_avg, \
                        windgust_max_std, \
                        windgust_mean_avg, \
                        windgust_mean_std, \
                        windgust_min_avg, \
                        windgust_min_std, \
                        windspeed_max_avg, \
                        windspeed_max_std, \
                        windspeed_mean_avg, \
                        windspeed_mean_std, \
                        windspeed_min_avg, \
                        windspeed_min_std) \
                        VALUES (%s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s, \
                                %s, %s, %s, %s, %s);'
    

    for idx, row in data.iterrows():
        if idx % 1000 == 0:
            print(idx)
        cur.execute(INSERT_STATEMENT, ( row['day'],
                                        row['country'],
                                        row['city'],
                                        row['precip_max_avg'],
                                        row['precip_max_std'],
                                        row['precip_mean_avg'],
                                        row['precip_mean_std'],
                                        row['specific_humidity_max_avg'],
                                        row['specific_humidity_max_std'],
                                        row['specific_humidity_mean_avg'],
                                        row['specific_humidity_mean_std'],
                                        row['specific_humidity_min_avg'],
                                        row['specific_humidity_min_std'],
                                        row['short_wave_radiation_max_avg'],
                                        row['short_wave_radiation_max_std'],
                                        row['short_wave_radiation_mean_avg'],
                                        row['short_wave_radiation_mean_std'],
                                        row['air_temperature_max_avg'],
                                        row['air_temperature_max_std'],
                                        row['air_temperature_mean_avg'],
                                        row['air_temperature_mean_std'],
                                        row['air_temperature_min_avg'],
                                        row['air_temperature_min_std'],
                                        row['windgust_max_avg'],
                                        row['windgust_max_std'],
                                        row['windgust_mean_avg'],
                                        row['windgust_mean_std'],
                                        row['windgust_min_avg'],
                                        row['windgust_min_std'],
                                        row['windspeed_max_avg'],
                                        row['windspeed_max_std'],
                                        row['windspeed_mean_avg'],
                                        row['windspeed_mean_std'],
                                        row['windspeed_min_avg'],
                                        row['windspeed_min_std']))

In [9]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [2]:
# loads data
weather = pd.read_pickle("to_upload/WORLD_20200101_20200430.pkl")

In [5]:
weather.to_pickle("to_upload/WORLD_20200101_20200430_PROTOCOL.pkl", protocol = 3)

In [ ]:
drop_weather_table(cur)
conn.commit()

create_weather_table(cur)
conn.commit()

import_data(cur, weather)
conn.commit()
print("data uploaded")

weather table deleted
weather table created
0
1000
2000
3000
4000
5000
6000
7000


## Test DB

In [ ]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [ ]:
cur.execute("""SELECT * FROM weather""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data.head()